# Решение задачи кредитного скоринга

В этом задании мы будем работать с данными клиентов банка. В отличие от задания с домами Бостона, данные в этой задаче не сразу готовы для построения модели, и нам потребуется предварительно посмотреть на данные и предобработать их.

Значение признаков:
* account: банковский счет (-1: отриц. баланс, 0: нет счета, 1: до 200 ед., 2: более 200 ед.)
* duration: на какой период запрашивают кредит
* credit_history: рейтинг по кредитной истории (от 0 - отличная кр. история до 4 - критическая ситуация с кредитами)
* amount: на какую сумму запрашивают кредит
* savings: сберегательный счет (0: нет, 1: < 100, 2: 100 <= ... < 500, 3: 500 <= ... < 1000, 4:  >= 1000)
* employment: срок работы на текущей позиции (0: не работает, 1: до 1 года, 2: от 1 до 4 лет, 3: от 4 до 7 лет, 4: более 7 лет)
* guarantors: 1 - есть поручители, 0 - нет
* residence:  сколько лет резидент
* age: возраст, деленный на 100
* credits_number: число кредитов
* job: 0: не работает, 1: неквалифицированный, 2: квалифицированный специалист, 3: высокая должность или бизнес
* maintenance_people: число людей, которых обеспечивает заемщик
* telephone: указан ли телефон (1: да, 0: нет)
* foreign: 1: иностранец, 0: нет
* real_estate: 1: есть недвижимость, 0: нет недвижимости
* life_insurance: 1: оформлено страхование жизни, 0, нет страховки
* car: 1: есть автомобиль, 0: нет автомобиля
* housing_rent: 1: платит за съем жилья, 0: не платит за съем жилья
* sex: пол - 1: муж., 0: жен.
* purpose: на какую цель запрашивают кредит (из нескольких вариантов)
* target: 1: кредит выдан, 0: в кредите отказано

Требуется решить задачу предсказания значения в последнем столбце, то есть задачу бинарной классификации.

По описанию данных понятно, что все признаки числовые (включая вещественные, порядковые, бинарные), кроме предпоследнего, который является категориальным.

In [ ]:
# эта ячейка устанавливает необходимые библиотеки (модули)
!pip install pandas
!pip install xlrd
!pip install sklearn
!pip install matplotlib

### 1. Загрузка, визуализация и предварительный анализ данных

Для данных кредитного скоринга нам понадобится выполнить больше предобработки данных, используя некоторое количество команд, которые мы ранее не разбирали. Эти команды не понадобятся в части для самостоятельной работы.

Используем модуль pandas и его функцию read_excel для загрузки данных:

In [ ]:
import pandas as pd
%matplotlib inline 
# служебная команда для графиков

In [ ]:
tab = pd.read_excel("scoring.xlsx")

Посмотрим на первые 10 строк с помощью функции head:

In [ ]:
tab.head(10)

По строкам - объекты (клиенты), по столбцам - признаки, последний столбец - целевая переменная (1 - кредит выдан, 0 - в кредите отказано).

Сколько объектов и признаков?

In [ ]:
tab.shape

Посмотрим, сколько кредитов выдано и не выдано:

In [ ]:
class_counts = tab["target"].value_counts()
class_counts

In [ ]:
class_counts.plot(kind='bar', title="Число выданных и не выданных кредитов")

Посмотрим общие статистики по данным:

In [ ]:
tab.describe()

* Есть ли пропуски в данных?
* Все ли признаки одного масштаба?

Посмотрим на гистограммы признаков:

In [ ]:
smth = tab.hist(figsize=(20, 20), bins=100)

* Признаки каких типов присутствуют в данных (вещественные, порядковые, категориальные, бинарные)?
* Есть ли в данных ненужные признаки (например, всегда равные одному значению)?

### 2. Подготовка данных

Выделим отдельно признаки и целевую переменную. Для первого этапа работы исключим категориальный признак "цель кредита", поэтому при создании X удалим последние два столбца.

In [ ]:
X = tab[tab.columns[:-2]]
y = tab["target"]

В предыдущем разделе мы увидели, что признаки имеют разный масштаб. Это помешает нам интерпретировать веса линейной модели, а также может усложнить обучение. Поэтому мы приведем признаки в один масштаб. Воспользуемся библиотекой sklearn:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
X[:3]

Теперь значения во всех столбцах от 0 до 1!

### Задания для самостоятельной работы

Теперь мы готовы к тому, чтобы повторить шаги, которые мы выполняли в задании с домами Бостона: разделение данных на обучение и контроль, построение и анализ модели.

При выполнении заданий используйте примеры кода из задания про дома Бостона.

Обязательно выполните все предыдущие ячейки. Для этого нажмите на эту ячейку, затем выберите в меню Cell -> Run all above.

#### Задача 1. Разделение данных на обучение и контроль

Используйте функцию train_test_split, чтобы разделить выборку X, y на обучение и контроль. Задайте test_size = 0.3, а также укажите аргумент random_state=1 (он понадобится нам в дальнейшем). 

Пример: train_test_split( .... test_size=0.3, random_state=1)

In [ ]:
from sklearn.model_selection import train_test_split
# функция для разделения выборки на обучающую и тестовую

In [ ]:
X_tr, X_te, y_tr, y_te  = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
X_tr.shape, X_te.shape

### 3. Обучение  модели

#### Задача 2.  Обучение линейной модели

Обучите линейную модель на обучающих данных. Используйте LogisticRegression, импортированную в следующей ячейке.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()
model.fit(X_tr, y_tr)

### 4. Получение предсказаний

#### Задача 3. Предсказания линейной модели

Выполните предсказания линейной модели на обучающей и тестовой выборказ.

In [ ]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

### 5. Оценивание качества модели

#### Задача 4. Качество линейной модели

Оцените качество линейной модели на обучающей и тестовой выборках. Воспользуемся метрикой accuracy и импортируем функцию accuracy_score. Эту функцию нужно использовать точно так же, как функцию mean_absolute_error из предыдущего задания, однако она оценивает качество классификации, а не регрессии. 

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Качество на обучении:", accuracy_score(y_tr, preds_tr))
print("Качество на контроле:", accuracy_score(y_te, preds_te))

### 6. Анализ модели

Выведем таблицу со столбцами "название признака" и "вес признака", отсортированную по значениям весов. Логичны ли вклады признаков?

In [ ]:
weights_data = {"веса":model.coef_[0],\
                "признаки": tab.columns[:-2]}
weights = pd.DataFrame(weights_data)
weights.sort_values("веса")

### 7. Улучшение модели

В самом начале работы мы исключили из данных категориальный признак "цель кредита". Давайте вернем его в наши данные и посмотрим, повысится ли качество модели, если мы грамотно обработаем этот признак. На лекции вы разобрали кодирование категориальных признаков, когда вместо одного категориального признака создают несколько бинарных, и у каждого объекта - только одна 1, остальные 0. Вернемся назад и выполним такое кодирование для наших данных. Для этого в модуле pandas функция get_dummies:

In [ ]:
tab_ohe = pd.get_dummies(tab, "purpose")

In [ ]:
tab_ohe.head()

В правой части данных появилось много 0 и 1.

Теперь повторяем все те же шаги, чтобы получить выборки:

In [ ]:
X_ohe = tab_ohe.drop("target", axis=1)
y = tab_ohe["target"]
scaler = MinMaxScaler()
X_ohe = scaler.fit_transform(X_ohe)
X_tr_ohe, X_te_ohe, y_tr, y_te  = train_test_split(X_ohe, y, \
                                             test_size=0.3, random_state=1)

Благодаря фиксации random_state=0 мы получаем одно и то же разделение.

#### Задача 5. Линейная модель с категориальным признаком

Повторите шаги задач 2-4 для новых данных: обучите линейную модель, выполните предсказания на обучающей и контрольной выборках и оцените качество (accuracy). Повысилось ли качество по сравнению с предыдущей моделью?

In [ ]:
model = LogisticRegression()
model.fit(X_tr_ohe, y_tr)

In [ ]:
preds_tr = model.predict(X_tr_ohe)
preds_te = model.predict(X_te_ohe)

In [ ]:
print("Качество на обучении:", accuracy_score(y_tr, preds_tr))
print("Качество на контроле:", accuracy_score(y_te, preds_te))

А что, если мы просто пронумеруем категории вместо "умного" кодирования? Для этого воспользуемся классом LabelEncoder:

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
tab["purpose"] = le.fit_transform(tab["purpose"])
X_le = tab[tab.columns[:-1]]
scaler = MinMaxScaler()
X_le = scaler.fit_transform(X_le)
X_tr_le, X_te_le, y_tr, y_te  = train_test_split(X_le, y, \
                                                     test_size=0.3,\
                                                     random_state=1)

#### Задача 6. Линейная модель с категориальным признаком - 2

Повторите шаги задач 2-4 для новых данных: обучите линейную модель, выполните предсказания на обучающей и контрольной выборках и оцените качество (accuracy). Изменилось ли качество по сравнению с двумя предыдущими моделями?

In [ ]:
model = LogisticRegression()
model.fit(X_tr_le, y_tr)

In [ ]:
preds_tr = model.predict(X_tr_le)
preds_te = model.predict(X_te_le)

In [ ]:
print("Качество на обучении:", accuracy_score(y_tr, preds_tr))
print("Качество на контроле:", accuracy_score(y_te, preds_te))

#### Задача 7. Подбор гиперпараметров

Чтобы улучшить качество, можно попробовать другие значения гиперпараметров. Будем использовать исходные данные X, y для простоты (без категориального признака).

Ключевой гиперпараметр логистической регрессии - коэффициент регуляризации C: LogisticRegression(C=...). Попробуйте значения от 0.001 до 1000, выберите лучшее значение с точки зрения каечствана тестовой выборке. Как и в задании с домами Бостона, можно выполнять упрощенную версию с копированием кода или усложненную версию с циклом. 

In [ ]:
import numpy as np
10.0**np.arange(-3, 4)